# Aygaz Görüntü İşleme Bootcampi

## Proje Hakkında
Bu projedeki amaç çeşitli hayvanları sınıflandıran bir CNN modeli kurmak. 

Bu proje kapsamında https://www.kaggle.com/datasets/rrebirrth/animals-with-attributes-2 veriseti kullanılmıştır.

Bu veri setinden 10 farklı hayvan sınıfı (collie, dolphin, elephant, fox, moose, rabbit, sheep, squirrel, giant panda ve polar bear) seçilmiş, görüntüler üzerinde veri ön işleme adımları gerçekleştirilmiştir. Model eğitildikten sonra, test seti ışık değişimleri ve bulanıklaştırma gibi manipülasyonlara tabi tutulmuş, ardından renk sabitliği algoritması uygulanarak performans değerlendirmesi yapılmıştır.

## Veri Setini Hazırlama
Burada projede kullanılacak olan 10 hayvan sınıfı belirlenmiş her sınıf için 650 adet fotoğraf seçilmiş, fotoğraflar 128x128 boyutuna ölçeklendirilmiş ve 0 ile 1 arasına normalize edilmiştir.

In [8]:
#gerekli kütüphaneler
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
#belirlenen hayvan sınıfları ve resim boyutu
selected_classes = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]
image_size = (128, 128)
image_count = 650

data_dir = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"

#veri setinin yüklenmesi
def load_data(data_dir, selected_classes, image_size, image_count):
    X, y = [], [] #X normalize edilmiş görüntüler, y ise etiketlerini tutar
    for class_index, class_name in enumerate(selected_classes):
        class_path = os.path.join(data_dir, class_name)
        
        if os.path.exists(class_path):
            images = os.listdir(class_path)[:image_count]
            for img_name in images:
                img_path = os.path.join(class_path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, image_size)
                    img = img / 255.0  # Normalizasyon edilir
                    X.append(img)
                    y.append(class_index)
        else:
            print("path not found")
    
    return np.array(X), np.array(y)

### eğitim ve test verilerine %70'e %30 olacak şekilde ayırma

In [7]:
X, y = load_data(data_dir, selected_classes, image_size, image_count)

# Veriyi eğitim %70 ve test %30 seti olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=9)

# Etiketleri kategorik hale getirme
y_train = to_categorical(y_train, num_classes=len(selected_classes))
y_test = to_categorical(y_test, num_classes=len(selected_classes))

print(f"Eğitim seti boyutu: Görüntüler => {X_train.shape}, Etiketler => {y_train.shape}")
print(f"Test seti boyutu: Görüntüler => {X_test.shape}, Etiketler => {y_test.shape}")

Eğitim seti boyutu: Görüntüler => (4550, 128, 128, 3), Etiketler => (4550, 10)
Test seti boyutu: Görüntüler => (1950, 128, 128, 3), Etiketler => (1950, 10)


## Modelin Eğitilmesi

Bu bölümde, model seçilen eğitim verisi üzerinde eğitilmiştir. Eğitim verilerinin çeşitliliğini artırmak için veri artırma teknikleri uygulanmıştır.

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Veri artırma işlevi
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

def create_model():
    model = Sequential([
        Input(shape=(128, 128, 3)),  
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(len(selected_classes), activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()

In [21]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_12 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,393,610 (28.20 MB)

 Trainable params: 7,393,610 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Modeli eğitme
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 75s 520ms/step - accuracy: 0.3892 - loss: 1.7244 - val_accuracy: 0.5051 - val_loss: 1.5019
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 73s 513ms/step - accuracy: 0.5242 - loss: 1.3961 - val_accuracy: 0.5303 - val_loss: 1.4087
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 74s 521ms/step - accuracy: 0.5932 - loss: 1.1876 - val_accuracy: 0.5595 - val_loss: 1.2960
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 74s 516ms/step - accuracy: 0.6594 - loss: 0.9632 - val_accuracy: 0.5938 - val_loss: 1.2194
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 74s 516ms/step - accuracy: 0.7448 - loss: 0.7633 - val_accuracy: 0.5918 - val_loss: 1.3116
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 73s 513ms/step - accuracy: 0.7951 - loss: 0.5866 - val_accuracy: 0.5913 - val_loss: 1.2894
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 74s 518ms/step - accuracy: 0.8222 - loss: 0.4845 - val_accuracy: 0.5969 - val_loss: 1.3828
Epoch 8/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 72s 504ms/step - accuracy: 0.8614 - loss: 0

## Modelin Test Edilmesi

In [24]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 120ms/step - accuracy: 0.5962 - loss: 1.4855
Test Loss: 1.5167063474655151, Test Accuracy: 0.5979487299919128


## Görüntülerin Manipüle Edilmesi

In [29]:
def get_manipulated_images(images):
    bright_images = []
    blurred_images = []
    for img in images:
        bright_img = cv2.convertScaleAbs(img, alpha=1.2, beta=10)
        blurred_img = cv2.GaussianBlur(img, (5, 5), 10)
        bright_images.append(bright_img)
        blurred_images.append(blurred_img)
    return bright_images, blurred_images

In [32]:
X_test_bright, X_test_blurred = get_manipulated_images(X_test)

## Modelin Manipüle Edilmiş Test Setleriyle Denenmesi

In [33]:
loss_bright, accuracy_bright = model.evaluate(X_test_bright, y_test)
print(f"Parlaklaştırılmış Test Doğruluğu: {accuracy_bright * 100:.2f}%")

61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 131ms/step - accuracy: 0.0891 - loss: 112.1712
Parlaklaştırılmış Test Doğruluğu: 9.54%


In [37]:
loss_blurred, accuracy_blurred = model.evaluate(X_test_blurred, y_test)
print(f"Bulanıklaştırılmış Test Doğruluğu: {accuracy_blurred * 100:.2f}%")

61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 132ms/step - accuracy: 0.4842 - loss: 1.7735
Bulanıklaştırılmış Test Doğruluğu: 48.67%


## Test setlerinin karşılaştırılması

In [38]:
print(f"Normal test seti için accuracy: {test_accuracy}")
print(f"Parlaklaştırılmış test seti için accuracy: {accuracy_bright}")
print(f"Bulanıklaştırılmış test seti için accuracy: {accuracy_blurred}")

Normal test seti için accuracy: 0.5979487299919128
Parlaklaştırılmış test seti için accuracy: 0.0953846126794815
Bulanıklaştırılmış test seti için accuracy: 0.4866666793823242
